In [1]:
import pandas as pd
import json
import requests
import ast
import yaml

In [30]:
def create_twitter_url():
    handle = "jessicagarson"
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/labs/2/tweets/search?{}&{}".format(mrf, q)
    return url

In [31]:
url = create_twitter_url()
print(url)

https://api.twitter.com/labs/2/tweets/search?max_results=100&query=from:jessicagarson


In [32]:
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

In [33]:
data = process_yaml()

In [34]:
def create_bearer_token(data):
    return data["search_tweets_api"]["bearer_token"]

In [35]:
bearer_token = create_bearer_token(data)

In [37]:
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [39]:
res_json = twitter_auth_and_connect(bearer_token, url)
print(res_json)

{'data': [{'id': '1265491805296119810', 'text': '@replylord What are goth roller skates?'}, {'id': '1265491691731116037', 'text': '@diskmem What are our stuffed baby Luigis doing when we aren’t looking.'}, {'id': '1265359178920534017', 'text': '@diskmem My favorite shirt🕵️\u200d♀️'}, {'id': '1265335302543101952', 'text': 'All of you complaining about cutting your own hair clearly never paid hundreds of dollars for a haircut reminiscent of the cuts you accidentally gave yourself in middle school.'}, {'id': '1265136167378399232', 'text': '@diskmem https://t.co/G0ChSIYlUD'}, {'id': '1264750444510920705', 'text': 'RT @seanmcque_n: It’s the golden age of bike riding in New York City'}, {'id': '1264410342194581506', 'text': '@armcannon We should stream / live preform together at another thing sometime soon'}, {'id': '1264249738951569410', 'text': 'We’re (@CharStiles and I) planning an Algorave in Minecraft. This is being put on by @livecodenyc and @wondervillenyc. Fill out this form if you a

In [40]:
def lang_data_shape(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)

In [41]:
documents = lang_data_shape(res_json)
print(documents)

{'documents': [{'id': '1265491805296119810', 'text': '@replylord What are goth roller skates?'}, {'id': '1265491691731116037', 'text': '@diskmem What are our stuffed baby Luigis doing when we aren’t looking.'}, {'id': '1265359178920534017', 'text': '@diskmem My favorite shirt🕵️\u200d♀️'}, {'id': '1265335302543101952', 'text': 'All of you complaining about cutting your own hair clearly never paid hundreds of dollars for a haircut reminiscent of the cuts you accidentally gave yourself in middle school.'}, {'id': '1265136167378399232', 'text': '@diskmem https://t.co/G0ChSIYlUD'}, {'id': '1264750444510920705', 'text': 'RT @seanmcque_n: It’s the golden age of bike riding in New York City'}, {'id': '1264410342194581506', 'text': '@armcannon We should stream / live preform together at another thing sometime soon'}, {'id': '1264249738951569410', 'text': 'We’re (@CharStiles and I) planning an Algorave in Minecraft. This is being put on by @livecodenyc and @wondervillenyc. Fill out this form if 

In [42]:
def connect_to_azure(data):
    azure_url = "https://week.cognitiveservices.azure.com/"
    language_api_url = "{}text/analytics/v2.1/languages".format(azure_url)
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    subscription_key = data["azure"]["subscription_key"]
    return language_api_url, sentiment_url, subscription_key

In [43]:
language_api_url, sentiment_url, subscription_key = connect_to_azure(data)

In [46]:
def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}

In [47]:
headers = azure_header(subscription_key)

In [49]:
def generate_languages(headers, language_api_url, documents):
    response = requests.post(language_api_url, headers=headers, json=documents)
    return response.json()

In [50]:
with_languages = generate_languages(headers, language_api_url, documents)
print(with_languages)

{'documents': [{'id': '1265491805296119810', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1265491691731116037', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1265359178920534017', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1265335302543101952', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1265136167378399232', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1264750444510920705', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1264410342194581506', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1264249738951569410', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1264023305188773888', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0

In [51]:
def combine_lang_data(documents, with_languages):
    langs = pd.DataFrame(with_languages["documents"])
    lang_iso = [x.get("iso6391Name")
                for d in langs.detectedLanguages if d for x in d]
    data_only = documents["documents"]
    tweet_data = pd.DataFrame(data_only)
    tweet_data.insert(2, "language", lang_iso, True)
    json_lines = tweet_data.to_json(orient="records")
    return json_lines

In [53]:
json_lines = combine_lang_data(documents, with_languages)
print(json_lines)

[{"id":"1265491805296119810","text":"@replylord What are goth roller skates?","language":"en"},{"id":"1265491691731116037","text":"@diskmem What are our stuffed baby Luigis doing when we aren\u2019t looking.","language":"en"},{"id":"1265359178920534017","text":"@diskmem My favorite shirt\ud83d\udd75\ufe0f\u200d\u2640\ufe0f","language":"en"},{"id":"1265335302543101952","text":"All of you complaining about cutting your own hair clearly never paid hundreds of dollars for a haircut reminiscent of the cuts you accidentally gave yourself in middle school.","language":"en"},{"id":"1265136167378399232","text":"@diskmem https:\/\/t.co\/G0ChSIYlUD","language":"en"},{"id":"1264750444510920705","text":"RT @seanmcque_n: It\u2019s the golden age of bike riding in New York City","language":"en"},{"id":"1264410342194581506","text":"@armcannon We should stream \/ live preform together at another thing sometime soon","language":"en"},{"id":"1264249738951569410","text":"We\u2019re (@CharStiles and I) pla

In [54]:
def add_document_format(json_lines):
    docu_format = '"' + "documents" + '"'
    json_docu_format = "{}:{}".format(docu_format, json_lines)
    docu_align = "{" + json_docu_format + "}"
    jd_align = json.dumps(docu_align)
    jl_align = json.loads(jd_align)
    return ast.literal_eval(jl_align)

In [55]:
document_format = add_document_format(json_lines)
print(document_format)

{'documents': [{'id': '1265491805296119810', 'text': '@replylord What are goth roller skates?', 'language': 'en'}, {'id': '1265491691731116037', 'text': '@diskmem What are our stuffed baby Luigis doing when we aren’t looking.', 'language': 'en'}, {'id': '1265359178920534017', 'text': '@diskmem My favorite shirt\ud83d\udd75️\u200d♀️', 'language': 'en'}, {'id': '1265335302543101952', 'text': 'All of you complaining about cutting your own hair clearly never paid hundreds of dollars for a haircut reminiscent of the cuts you accidentally gave yourself in middle school.', 'language': 'en'}, {'id': '1265136167378399232', 'text': '@diskmem https:\\/\\/t.co\\/G0ChSIYlUD', 'language': 'en'}, {'id': '1264750444510920705', 'text': 'RT @seanmcque_n: It’s the golden age of bike riding in New York City', 'language': 'en'}, {'id': '1264410342194581506', 'text': '@armcannon We should stream \\/ live preform together at another thing sometime soon', 'language': 'en'}, {'id': '1264249738951569410', 'text

In [56]:
def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(
        sentiment_url, headers=headers, json=document_format)
    return response.json()

In [57]:
sentiments = sentiment_scores(headers, sentiment_url, document_format)
print(sentiments)

{'documents': [{'id': '1265491805296119810', 'score': 0.5}, {'id': '1265491691731116037', 'score': 0.286409854888916}, {'id': '1265359178920534017', 'score': 0.9353771209716797}, {'id': '1265335302543101952', 'score': 0.12720105051994324}, {'id': '1265136167378399232', 'score': 0.7015109062194824}, {'id': '1264750444510920705', 'score': 0.9699492454528809}, {'id': '1264410342194581506', 'score': 0.8423745632171631}, {'id': '1264249738951569410', 'score': 0.7769913673400879}, {'id': '1264023305188773888', 'score': 0.7277792096138}, {'id': '1263952732785958914', 'score': 0.1218404471874237}, {'id': '1263950822574100481', 'score': 0.706977128982544}, {'id': '1263948281790566402', 'score': 0.9295811653137207}, {'id': '1263944783648743426', 'score': 0.5}, {'id': '1263907060300816390', 'score': 0.863063633441925}, {'id': '1263691672396890113', 'score': 0.8640784025192261}, {'id': '1263684215243235328', 'score': 0.5}], 'errors': []}


In [65]:
def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()

In [66]:
week_score = mean_score(sentiments)
print(week_score)

0.6470708809792995


In [67]:
def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("You had a positve week")
    elif week_score > 0.45 or week_score == 0.45:
        print("You had a neautral week")
    else:
        print("You had a negative week, I hope it gets better")

In [69]:
week_logic(week_score)

You had a neautral week
